## Imports

In [ ]:
import pandas as pd
import numpy as np
import pypsa
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import plotly
import datetime
import seaborn as sns
import ast

from utils import market_values, market_values_by_time_index

In [ ]:
n =pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")

In [ ]:
n.storage_units_t.p

In [ ]:
# overall generation: active power at bus (positive if net generation at bus) in MW
n.buses_t.p

In [ ]:
n.generators_t.p_max_pu

In [ ]:
lmp_regions = n.buses_t.marginal_price.loc[:, n.buses.location.unique()[:-1]]
lmp_regions.columns

## Price Duration curve

In [ ]:
# price duration curve per region
plt.figure(figsize=(8, 6))
plt.plot(lmp_regions['SK0 0'].sort_values(ascending=False).values)
# plt.ylim(-100, 500)
plt.xlabel("Hours of the year in 3h steps (sorted)")
plt.ylabel("Market clearing price in EUR/MWh)")
plt.show()

# would be interesting to see which technology is the price setter in the respecting hour
# calculating area under the curve to see how much investment is in the market?

In [ ]:
# overall price duration curve (mean aggregation)
plt.figure(figsize=(12, 8))
plt.plot(lmp_regions.mean(axis=1).sort_values(ascending=False).values)
# plt.ylim(-100, 500)
plt.show()

- constructing figure from paper Böttger & Härtel
- Figure 17: Power generation and consumption and market values (or capture prices) of individual technologies in the wholesale electricity market of Germany in 2050 for the 28 countries scenario, own illustration based on own computations.
- effect of storage technologies on renewables curtailment (calculate how much energy the storages take from the market that is from a renewable source)
- check wether the market values of renewables lie below the average market value

#### Merit Order Single Case (Example)

In [ ]:
# example of bus and snapshot
bus = "AL0 0" # "DE0 1"
snapshot = "2013-01-02 00:00:00"
buses = n.buses[n.buses.location == bus].index
# 	active power at bus (positive if net generation at bus) in MW
n.buses_t.p.loc[snapshot,buses]

## Generators

In [ ]:
# The maximum output for each snapshot per unit of p_nom for the OPF (e.g. for variable renewable generators this can change due to weather conditions; for conventional generators it represents a maximum dispatch).
# p_max_pu is 1 for all generators (for VRE it has the be replaced by the corresponding value from n.generators_t
n.generators.p_max_pu

In [ ]:
# Nominal power for limits in OPF.
# only ror has nominal power limit that is not 0 -> limits in p_nom and p_nom_opt of ror are identical -> use p_nom_opt as nominal power limit for all generators
n.generators.p_nom_opt

In [ ]:
# gas and oil are not in n.generators_t.p_max_pu.columns: their maximum output is static / not variable
# and determined by n.generators.p_nom_opt*n.generators.p_max_pu(=1) in every time step
n.generators.index.difference(pd.Index(n.generators_t.p_max_pu.columns))

In [ ]:
# calculate the possible output for the generators for every time step (first without gas and oil)
# TODO: Does this already include the state of load for e.g. storage units? Maybe also relevant for ror?
max_out_gen = n.generators_t.p_max_pu * n.generators.p_nom_opt[n.generators_t.p_max_pu.columns]
max_out_gen

In [ ]:
# real output
n.generators_t.p[n.generators_t.p_max_pu.columns]

In [ ]:
# compare to real output
out_ratio = n.generators_t.p[n.generators_t.p_max_pu.columns] / max_out_gen
out_diff = max_out_gen -n.generators_t.p[n.generators_t.p_max_pu.columns]

In [ ]:
out_ratio#.describe()
#out_diff

### Results

In [ ]:
# indicate which generator is marginal generator
th_ratio = 0.95
th_prod_rel = 0.05
th_prod_abs = 10

# ratio has to be lower than threshold
out_ratio[out_ratio < th_ratio] = 0.5
# output has to be greater than threshold (relatively to max_output) and greater than th_prod_abs MW
out_prod = n.generators_t.p[n.generators_t.p_max_pu.columns].copy()
out_prod[(out_prod > (max_out_gen*th_prod_rel)) & (out_prod > th_prod_abs)] = 0.5
out_incidence = out_ratio + out_prod
out_incidence[out_incidence != 1] = 0
out_incidence = out_incidence.astype(int)
out_incidence

In [ ]:
# number can be higher than 181 as there are also generators for other technologies not just electricity -> maybe first only concentrate on AC (main bus)
plt.plot(out_incidence.sum(axis=1))
out_incidence.sum(axis=1).max()

In [ ]:
# check results
gen = "SK0 0 ror" # "AT0 1 onwind" # "SK0 0 solar rooftop" # "DE0 10 solar" # AL0 0 solar, AT0 1 onwind
start = 0
end = 100

plt.figure(figsize=(20, 8))
plt.plot(n.generators_t.p[gen][start:end], "o", color="green", label="real generation")
plt.plot(max_out_gen[gen][start:end], "x", color="black", label="max generation")
plt.plot(out_incidence[gen][start:end]*max_out_gen[gen][start:end].mean(), "x", color="red", marker="*", label="marginal generator indicator")
plt.legend()
plt.show()

### Problems
- for very low possible VRE output, the indication is often set but, the technology is surely not the marginal generator (check AT0 1 onwind) first 3 hours (corrected by adding constraint for minimum output of 10 MW)


## Storage Units

In [ ]:
# Challenge1: comparing only the actual output with the product of nominal power and maximum output is not sufficient as the storage can be empty / not sufficiently full ?
# Challenge2: the real output can be either positive or negative (when storing energy) -> no problem as threshols only takes values for positive generation
# n.storage_units.p_nom == n.storage_units.p_nom_opt is True -> you can use both
# n.storage_units.index.difference(pd.Index(n.storage_units_t.p.columns)) -> no difference in indices

In [ ]:
n.storage_units.index

In [ ]:
# calculate the possible output for the storage unit for every time step (all static as n.storage_units_t.p_max_pu is empty)
max_output = n.storage_units.p_max_pu * n.storage_units.p_nom_opt
# max_output = pd.DataFrame(max_output.values.reshape(1,170), columns=max_output.index)
# max_output

In [ ]:
n.storage_units_t.p_max_pu

In [ ]:
# real output
n.storage_units_t.p

In [ ]:
# state of charge (State of charge as calculated by the OPF in MWh)
n.storage_units_t.state_of_charge

In [ ]:
max_output_ts = n.storage_units_t.p.copy()
for snap in n.storage_units_t.p.index:
    max_output_ts.loc[snap] = max_output[n.storage_units_t.p.columns]
max_output_ts

In [ ]:
# take minimum of state of charge and max_output_ts
# Problem: There are a lot of entries where the ration is larger than 1 if you take the minimum; THis is not the case for skipping this step (maybe skip for now)
# max_output_ts = np.minimum(max_output_ts, n.storage_units_t.state_of_charge)
max_output_ts

In [ ]:
# compare to real output
out_ratio = n.storage_units_t.p / max_output_ts
out_diff = max_output_ts -  n.storage_units_t.p

In [ ]:
out_ratio

In [ ]:
#out_diff

### Results

In [ ]:
# indicate which storage_unit is marginal storage_unit
th_ratio = 0.95
th_prod_rel = 0.05
th_prod_abs = 10

# ratio has to be lower than threshold
out_ratio[out_ratio < th_ratio] = 0.5 # th_ratio
# output has to be greater than threshold (relatively to max_output) and greater than th_prod_abs MW
out_prod = n.storage_units_t.p.copy()
out_prod[(out_prod > (max_output_ts*th_prod_rel)) & (out_prod > th_prod_abs)] = 0.5 # th_prod
out_incidence_su = out_ratio + out_prod
out_incidence_su[out_incidence_su != 1] = 0
out_incidence_su = out_incidence_su.astype(int)
out_incidence_su

In [ ]:
# number cannot be higher than 181 as PHS and hydro both feed into the AC bus (they lie at the bus directly)
plt.plot(out_incidence_su.sum(axis=1))
out_incidence_su.sum(axis=1).max()

In [ ]:
# check results
su = "BG0 1 PHS" # "BG0 1 hydro"; BG0 1 PHS
start = 0
end = 100

plt.figure(figsize=(20, 8))
plt.plot(n.storage_units_t.p[su][start:end], "o", color="green", label="real generation")
plt.plot(max_output_ts[su][start:end], "x", color="black", label="max generation")
plt.plot(out_incidence_su[su][start:end]*max_output_ts[su][start:end].mean()*0.75, ls="", color="red", marker="*", label="marginal generator indicator")
plt.legend()
plt.show()

## Links
Challenges:
- Links with multiple outputs: Links can also be defined with multiple outputs in fixed ratio to the power in the single input by defining new columns bus2, bus3, etc. (bus followed by an integer) in network.links along with associated columns for the efficiencies efficiency2, efficiency3, etc. The different outputs are then equal to the input multiplied by the corresponding efficiency;
- How to handle links with multiple outputs?
- Links are connected to stores which might be empty / not sufficiently full!

In [ ]:
# specify method to determine marginal generator: 1 means using the p_max_pu * p_nom_opt as maximum capacity and 2 means using the maximum real output as capcity
method = 2 #

In [ ]:
# n.links.index.difference(pd.Index(n.links_t.p0.columns)) is empty -> no difference in indices

In [ ]:
# Active power at bus0 (positive if branch is withdrawing power from bus0)
# only negative values would be relevant here (negative means generating energy)
# TODO: negative values only exist for the electricity distribution grid (skip for now)
n.links_t.p0[n.links_t.p0 < 0].sum()

In [ ]:
# flip sign to make generation poitive
n_links_p1 = n.links_t.p1 *-1
n_links_p1

### Determine maximum possible output

In [ ]:
# Limit of active power which can pass through link (n.links.p_nom can not be used)
# Optimised nominal power
n.links.p_nom_opt.head()

In [ ]:
# some links have a static p_max_pu value and some have an alternating (series)
index_series_li = n.links_t.p_max_pu.columns
index_static_li = n.links.index.difference(n.links_t.p_max_pu.columns)

In [ ]:
# calculate the possible output for the link for every time step

if method == 1:
    # use p_max_pu * p_nom_op
    max_output_links_static = n.links.loc[index_static_li].p_max_pu * n.links.loc[index_static_li].p_nom_opt
elif method == 2:
    # alternatively use the maximum of the real output and set it as the maximum capacity
    max_output_links_static= n_links_p1[index_static_li].max()

In [ ]:
# make ts of max_output_links_static
max_output_links_ts_static = n.links_t.p0[index_static_li].copy()
for snap in n.links_t.p0.index:
    max_output_links_ts_static.loc[snap] = max_output_links_static[index_static_li]

In [ ]:
# calc time series of time dependent p_max_pu links
max_output_links_ts_series = n.links_t.p_max_pu * n.links.p_nom_opt[index_series_li]

In [ ]:
# merge static and series values and reorder columns
max_output_links_ts = pd.concat([max_output_links_ts_static, max_output_links_ts_series], axis=1)[n.links_t.p0.columns]
max_output_links_ts

In [ ]:
#TODO: add step to check the charging state of the store

In [ ]:
# compare to real output
out_ratio_links = n_links_p1 / max_output_links_ts
out_diff_links = max_output_links_ts - n_links_p1

In [ ]:
out_ratio_links.describe()

### Results

In [ ]:
# indicate which link is marginal generator for p1
th_ratio = 0.95
th_prod_rel = 0.05
th_prod_abs = 10

# ratio has to be lower than threshold
out_ratio_links[out_ratio_links < th_ratio] = 0.5 # th_ratio
# output has to be greater than threshold (relatively to max_output) and greater than th_prod_abs MW
out_prod_links = n_links_p1.copy()
out_prod_links[(out_prod_links > (max_output_links_ts*th_prod_rel)) & (out_prod_links > th_prod_abs)] = 0.5 # th_prod
out_incidence_li = out_ratio_links + out_prod_links
out_incidence_li[out_incidence_li != 1] = 0
out_incidence_li = out_incidence_li.astype(int)
out_incidence_li

In [ ]:
out_incidence_li.describe()

In [ ]:
# number can be higher than 181 as there are also generators for other technologies not just electricity -> maybe first only concentrate on AC (main bus)
plt.plot(out_incidence_li.sum(axis=1))
out_incidence_li.sum(axis=1).max()

In [ ]:
# check results
li = "AL0 0 urban central solid biomass CHP CC" # "AL0 0 residential rural ground heat pump" #DE0 0 H2 Fuel Cell" # "AL0 0 urban central solid biomass CHP CC"# "DE0 0 H2 Fuel Cell" # "SE3 6 urban central resistive heater"
start = 0
end = 2920

plt.figure(figsize=(20, 8))
plt.plot(n_links_p1[li][start:end], "o", color="green", label="real generation")
plt.plot(max_output_links_ts[li][start:end], ":", color="black", label="max generation")
plt.plot(out_incidence_li[li][start:end]*max_output_links_ts[li][start:end].mean()*0.75, ls="", color="red", marker="*", label="marginal generator indicator")
plt.title(f"{li}")
plt.legend()
plt.show()

### Problems
- For "AL0 0 urban central solid biomass CHP CC" the max generation is not the real maximum possible output; same for "AL0 0 urban central solid biomass CHP" -> this can be obtained as the real generation never reaches the max generation (same for "DE0 0 H2 Fuel Cell")
- For a lot of links the real ouptut is higher thant he maximum output: "AL0 0 residential rural ground heat pump"

Possible solution:
- use the maximum of the real output and the theoretical maximum output for the static links


In [ ]:
# check if maximum output is ever reached (only check for static links)
ratio = pd.DataFrame(n_links_p1[index_static_li].max() / max_output_links_static).rename(columns={0: "data"})

In [ ]:
# this is 0 if you use the real maximum output as capacity (method2)
ratio[ratio.data > 1]

In [ ]:
# 4757 for method 1; 0 for method 2
ratio[ratio.data < 0.9]

In [ ]:
# good cases
# 2156 for method 1; 0 for method 2
ratio[(ratio.data > 0.9) & (ratio.data <= 1)]

## Determine marginal generator

In [ ]:
assert 0

In [ ]:
%%time
# create empty df (takes 2 mins)
mg = pd.DataFrame(index=n.buses_t.p.index, columns=n.buses.index)
for col in mg.columns:
    mg[col] = [[] for _ in range(len(mg))]
mg

In [ ]:
%%time
# assign marginal generator for every time step and carrier bus (takes <20 mins)
for snap in out_incidence.index:
    for gen in out_incidence.columns:
        if out_incidence.loc[snap,gen] == 1:
            mg.loc[snap,n.generators.bus[gen]].append(gen)
    for su in  out_incidence_su.columns:
        if out_incidence_su.loc[snap,su] == 1:
            mg.loc[snap,n.storage_units.bus[su]].append(su)
    for li in  out_incidence_li.columns:
            if out_incidence_li.loc[snap,li] == 1:
                mg.loc[snap,n.links.bus1[li]].append(li)

In [ ]:
# save
mg.to_csv("../data/processed/mg_gen_su_li_v1_method2.csv")

In [ ]:
# read back with snapshots as Datetime Index and Cells as lists
mg_back_m1 = pd.read_csv(filepath_or_buffer="../data/processed/mg_gen_su_li_v1_method1.csv", index_col="snapshot", parse_dates=True, converters={col: ast.literal_eval for col in n.buses.index})
mg_back_m2 = pd.read_csv(filepath_or_buffer="../data/processed/mg_gen_su_li_v1_method2.csv", index_col="snapshot", parse_dates=True, converters={col: ast.literal_eval for col in n.buses.index})

In [ ]:
mg = mg_back_m1
# mg = mg_back_m2

In [ ]:
mg.head()

#### Plot for one bus

In [ ]:
bus = "AL0 0" # "AL0 0" # "DE0 0"
df = pd.concat([mg[bus], n.buses_t.p[bus], n.buses_t.marginal_price[bus]], axis=1)
df.columns = [f"{bus}_mg", f"{bus}_p", f"{bus}_lmp"]
df[f"{bus}_mg labels"] = df[f"{bus}_mg"].astype(str)
df

In [ ]:
# lmp per marginal generator
plt.figure(figsize=(8, 6))
for label in df[f"{bus}_mg"].astype(str).unique():
    plt.plot(df[df[f"{bus}_mg labels"] == label][f"{bus}_lmp"], "x", label=label)

# plt.ylim(-100, 500)
plt.xlabel("Hours of the year in 3h steps (sorted)")
plt.ylabel("Market clearing price in EUR/MWh)")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

# capa nochmal mit effizienf falls nicht nur strom produziert wird (

In [ ]:
df2 = df.sort_values(by=[f"{bus}_lmp"], ascending=False)
df2.index = np.arange(1, len(df)+1)

# price duration curve per region
plt.figure(figsize=(16, 8))
for label in df[f"{bus}_mg"].astype(str).unique():
    plt.plot(df2[df2[f"{bus}_mg labels"] == label][f"{bus}_lmp"], ".", label=label)

# plt.ylim(-100, 500)
plt.xlabel("Hours of the year in 3h steps (sorted)")
plt.ylabel("Market clearing price in EUR/MWh)")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),fancybox=True, shadow=True, ncol=2)
plt.show()

In [ ]:
# get counts of marginal generators and labels
# labels
df2[f"{bus}_mg labels"].value_counts()

In [ ]:
# single marginal generators
mgs = []
for item in df2[f"{bus}_mg"]:
    mgs.extend(item)
mgs = set(mgs)
mgs

In [ ]:
# single counts / counts of marginal generators
df2[f"{bus}_mg"].apply(','.join).astype(str).str.get_dummies(sep=',').sum().sort_values(ascending=False)

In [ ]:
assert 0

## Determine marginal price

### Investigate case
-> power_balance notebook

In [ ]:
mg_back

In [ ]:
n.buses.loc[pd.Index(n.stores.bus.values)].location

In [ ]:
pd.Index(n.stores.bus.values)

In [ ]:
pd.Index(n.stores.bus.values)

In [ ]:
n.stores.bus.values

### ToDO

- Für einzelne Fälle rausfinden welche die marginal generators sind und deren lmp nachvollziehen. Dann damit die anderen marginal generator berechnen
- idea: if the net flow away from a bus is positive this means that the marginal generator has to be at the bus? -> no, only if all flows away from the bus are positive

In [ ]:
# Lines (what reaches bus?)
# 1. sum of power @ lines with bus as bus0 and p0 (Active power at bus0 (positive if branch is withdrawing power from bus0)
n.lines_t.p0[n.lines[n.lines.bus0 == bus].index].sum(axis=1)
# 2. sum of power @ lines with bus as bus1 and p1 (Active power at bus1 (positive if branch is withdrawing power from bus1)
n.lines_t.p1[n.lines[n.lines.bus1 == bus].index].sum(axis=1)
# sum (if negative power is feed into bus)
lines = n.lines_t.p0[n.lines[n.lines.bus0 == bus].index].sum(axis=1) + n.lines_t.p1[n.lines[n.lines.bus1 == bus].index].sum(axis=1)
lines[snapshot]

In [ ]:
# loads
loads_index = n.loads[[s in buses for s in n.loads.bus]].bus.index
# active power at bus (positive if net load) in MW
n.loads_t.p.loc[snapshot, loads_index]#.sum()

In [ ]:
# generation in whole region
gens_index = n.generators[[s in buses for s in n.generators.bus]].bus.index
# active power at bus (positive if net generation) in MW
n.generators_t.p.loc[snapshot, gens_index]#.sum(axis=1)

In [ ]:
# generation directly connected to bus
gens_index_direct = n.generators[n.generators.bus == bus].bus.index
n.generators_t.p.loc[snapshot, gens_index_direct]#.sum(axis=1)

In [ ]:
# storages
storage_index = n.storage_units[[s in buses for s in n.storage_units.bus]].bus.index
# active power at bus (positive if net generation) in MW
n.storage_units_t.p.loc[snapshot, storage_index]#.sum(axis=1)

In [ ]:
# storage directly at bus
storage_index_direct = n.storage_units[n.storage_units.bus == bus].bus.index
n.storage_units_t.p.loc[snapshot, storage_index_direct]#.sum(axis=1)

In [ ]:
# stores
stores_index = n.stores[[s in buses for s in n.stores.bus]].bus.index
# active power at bus (positive if net generation) in MW
n.stores_t.p.loc[snapshot, stores_index]#.sum(axis=1)

In [ ]:
# stores directly at bus
stores_index_direct = n.stores[n.stores.bus == bus].bus.index
# active power at bus (positive if net generation) in MW
n.stores_t.p.loc[snapshot, stores_index_direct]#.sum(axis=1)

In [ ]:
# link

In [ ]:
n.links[n.links.bus0 == bus]

In [ ]:
n.links[n.links.bus1 == bus]

In [ ]:
# Links (what reaches bus?)
# 1. sum of power @ links with bus as bus0 and p0 (Active power at bus0 (positive if branch is withdrawing power from bus0)
n.links_t.p0[n.links[n.links.bus0 == bus].index].sum(axis=1)
# 2. sum of power @ links with bus as bus1 and p1 (Active power at bus1 (positive if branch is withdrawing power from bus1)
n.links_t.p1[n.links[n.links.bus1 == bus].index].sum(axis=1)
# sum (if negative power is feed into bus)
links_sum = n.links_t.p0[n.links[n.links.bus0 == bus].index].sum(axis=1) + n.links_t.p1[n.links[n.links.bus1 == bus].index].sum(axis=1)
links_sum[snapshot]

In [ ]:
n.links_t.p0.loc[snapshot , n.links[n.links.bus0 == bus].index]

In [ ]:
n.links_t.p1.loc[snapshot , n.links[n.links.bus1 == bus].index]

In [ ]:
# aggregation for bus0
# balance of electricity feed in and withdrawal via lines
li = lines[snapshot]
# load at direct bus (not other technologies)
lo = n.loads_t.p.loc[snapshot, loads_index][bus].sum()
# generation at direct bus (not other technologies)
gen = n.generators_t.p.loc[snapshot, gens_index_direct].sum()
lk = links_sum[snapshot]

gen - li - lo - lk

In [ ]:
n.loads_t.p.loc[snapshot, loads_index][bus]

In [ ]:
n.buses_t.marginal_price[buses]

In [ ]:
n.stores

In [ ]:
# 17 buses in region AL0 0
n.buses[n.buses.location == "AL0 0"]

In [ ]:
# 181 regions and EU
n.buses.location.unique()

In [ ]:
n.loads[['AL0 0' in s for s in n.loads.index]]

In [ ]:
n.loads.bus.unique()

In [ ]:
n.generators.bus.unique()

In [ ]:
n.storage_units.bus

In [ ]:
n.stores.bus

In [ ]:
n.loads_t.p

In [ ]:
n.loads_t.p.loc["2013-01-02 03:00:00", ['AL0 0' in s for s in n.loads_t.p.columns]]

In [ ]:
n.loads_t.p

In [ ]:
n.loads_t.p_set.loc["2013-01-02 00:00:00", ['AL0 0' in s for s in n.loads_t.p_set.columns]]

In [ ]:
n.buses[['AL0 0' in s for s in n.buses.index]]

In [ ]:
# active power at bus (positive if net generation at bus) (MW)
n.buses_t.p
# 608 MWh missing at ALO 0 that are feed in from other buses / nodes via lines 1,2,3,4
1639.391422 - 1053.551832 - 269.964278

In [ ]:
# marginal cost of generator in currency/MWh
n.generators[['AL0 0' in s for s in n.generators.index]]

In [ ]:
# actual dispatch of the generator
n.generators_t.p.loc["2013-01-02 00:00:00", ['AL0 0' in s for s in n.generators_t.p.columns]]

In [ ]:
n.storage_units[['AL0 0' in s for s in n.storage_units.index]]

In [ ]:
n.storage_units.marginal_cost.describe()

In [ ]:
# Locational marginal price from LOPF from power balance constraint (currency/MWh)
plt.plot(n.buses_t.marginal_price['AL0 0'])

In [ ]:
n.stores[['AL0 0' in s for s in n.stores.index]]

In [ ]:
# active power at bus (positive if net generation) (MW)
n.stores_t.p.loc["2013-01-02 00:00:00", ['AL0 0' in s for s in n.stores_t.p.columns]]

In [ ]:
n.stores.marginal_cost[['AL0 0' in s for s in n.stores.index]]

In [ ]:
n.generators_t.p

In [ ]:
n.lines

In [ ]:
n.lines_t.p0[n.lines[n.lines.bus0 == "AL0 0"].index].sum(axis=1)

In [ ]:
# Active power at bus0 (positive if branch is withdrawing power from bus0).
n.lines_t.p0[["1","2","3","4"]].sum(axis=1)
# -608.163407 is feed into bus ALO 0
# 1053.552001

In [ ]:
n.lines_t.p1[["1","2","3"]].sum(axis=1)

In [ ]:
# no transformers
n.transformers_t

In [ ]:
n.storage_units

In [ ]:
n.buses_t.marginal_price

In [ ]:
import plotly.offline as py
from plotly.graph_objs import *

n.iplot()

In [ ]:
n.buses

In [ ]:
# reconstruction of electricity balance at bus 'AL0 0' at time "2013-01-02 00:00:00"
date = "2013-01-02 00:00:00"
bus = 'AT0 0' # "AT0 0 hydro

# load
load = n.loads_t.p_set.loc[date, [bus]]

# generation
gen = n.generators_t.p.loc[date, [bus in s for s in n.generators_t.p.columns]].sum()

# storage
storage = n.storage_units_t.p.loc[date, [bus in s for s in n.storage_units_t.p.columns]].sum()

# stores
# n.stores_t.p.loc[date, [bus in s for s in n.stores_t.p.columns]]

# lines
lines = n.lines_t.p0.loc[date, n.lines[n.lines.bus0 == bus].index].sum()

# balance
load - gen - storage - lines

In [ ]:
n.lines_t.p1[n.lines[n.lines.bus1 == bus].index].sum(axis=1)

In [ ]:
n.stores.bus

In [ ]:
# CO2 Preis

## Backup

## Stores

In [ ]:
n.stores

In [ ]:
# Challenge1: comparing only the actual output with the product of nominal power and maximum output is not sufficient as the storage can be empty / not sufficiently full ?
# Challenge2: the real output can be either positive or negative (when storing energy)
# n.stores.e_nom always zero -> use n.stores.e_nom_opt
# n.stores.index.difference(pd.Index(n.stores_t.p.columns)) -> no difference in indices

In [ ]:
n.stores.e_nom_min

In [ ]:
# calculate the possible energy capacity for the generators for every time step (not really relevant for our use case) -> maximum output power cannot really be calculated
max_output_e = n.stores.e_max_pu * n.stores.e_nom_opt
max_output_e

In [ ]:
# real output
n.stores_t.p

In [ ]:
# set maximum output as maximum possible output
max_output_ts = n.stores_t.p.copy()
for snap in n.stores_t.p.index:
    max_output_ts.loc[snap] = n.stores_t.p[n.stores_t.p.columns].max()
max_output_ts

In [ ]:
# compare to real output
out_ratio = n.stores_t.p / max_output_ts
out_ratio

In [ ]:
# indicate which store is marginal store

# ratio has to be lower than threshold
out_ratio[out_ratio < th_ratio] = 0.5 # th_ratio
# output has to be greater than threshold (relatively to max_output)
out_prod = n.stores_t.p.copy()
out_prod[out_prod > (max_output_ts*th_prod)] = 0.5 # th_prod
out_incidence_st = out_ratio + out_prod
out_incidence_st[out_incidence_st != 1] = 0
out_incidence_st = out_incidence_st.astype(int)
out_incidence_st

#### Results

In [ ]:
# number can be higher than 181 as there are also generators for other technologies not just electricity
plt.plot(out_incidence_st.sum(axis=1))
out_incidence_st.sum(axis=1).max()

In [ ]:
# check results
st = "DE0 20 home battery" # DE0 11 H2 Store, DE0 20 home battery, DE0 15 H2 Store
start = 0
end = 200

plt.figure(figsize=(20, 8))
plt.plot(n.stores_t.p[st][start:end], "o", color="green", label="real generation")
plt.plot(max_output_ts[st][start:end], "x", color="black", label="max generation")
plt.plot(out_incidence_st[st][start:end]*max_output_ts[st][start:end].mean()*0.75, "x", color="red", marker="*", label="marginal generator indicator")
plt.legend()
plt.show()